Code by AAKASH DESHPANDE

Import Libraries

In [19]:
# Remove conflicting packages from the Kaggle base environment.
!pip uninstall -qqy kfp jupyterlab libpysal thinc spacy fastai ydata-profiling google-cloud-bigquery google-generativeai
# Install langgraph and the packages used in this lab.
!pip install -qU 'langgraph==0.3.21' 'langchain-google-genai==2.1.2' 'langgraph-prebuilt==0.1.7'

ERROR: Invalid requirement: "'langgraph==0.3.21'": Expected package name at the start of dependency specifier
    'langgraph==0.3.21'
    ^


In [17]:
from google import genai
from google.genai import types
import langgraph
from IPython.display import HTML, Markdown, display

Set up a retry helper. This allows you to "Run all" without worrying about per-minute quota.

In [2]:
from google.api_core import retry


is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

In [ ]:
GOOGLE_API_KEY = " "

In [16]:
client = genai.Client(api_key=GOOGLE_API_KEY)

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="Explain who is the friend of farmer")

Markdown(response.text)

The "friend of the farmer" is a term commonly used to describe animals, insects, or practices that are beneficial to agriculture and farming. There isn't a single, definitive "friend," but rather a collection of things that contribute to a farmer's success. Here are some key examples:

*   **Earthworms:** They aerate the soil, improve drainage, and enrich it with their castings, making nutrients more accessible to plants.

*   **Bees and other Pollinators:** Bees, butterflies, and other insects are essential for pollinating crops, which is vital for fruit and seed production.

*   **Ladybugs:** They are voracious predators of aphids and other harmful pests, helping to protect crops without the need for chemical pesticides.

*   **Certain Birds:** Some birds consume insects and weed seeds, helping to control pests and weeds naturally.

*   **Beneficial Bacteria and Fungi:** Microorganisms in the soil can help plants absorb nutrients, protect them from diseases, and improve soil health.

*   **Crop Rotation and Cover Cropping:** These farming practices improve soil health, reduce pest and disease problems, and can increase crop yields.

*   **Livestock:** Livestock can provide manure for fertilizer, help control weeds, and can be integrated into crop rotations for a more sustainable farming system.

In short, **anything that helps a farmer grow healthy crops, control pests, improve soil health, and increase yields can be considered a "friend of the farmer."** The concept highlights the importance of biodiversity and sustainable farming practices.

In [7]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph.message import add_messages


class OrderState(TypedDict):
    """State representing the customer's order conversation."""

    # The chat conversation. This preserves the conversation history
    # between nodes. The `add_messages` annotation indicates to LangGraph
    # that state is updated by appending returned messages, not replacing
    # them.
    messages: Annotated[list, add_messages]

    # The customer's in-progress order.
    order: list[str]

    # Flag indicating that the order is placed and completed.
    finished: bool


# The system instruction defines how the chatbot is expected to behave and includes
# rules for when to call different functions, as well as rules for the conversation, such
# as tone and what is permitted for discussion.
BARISTABOT_SYSINT = (
    "system",  # 'system' indicates the message is a system instruction.
    "You are a FarmaBot, an interactive crop consultant system. A human will talk to you about the topics related to farmer"
    "Persona: You are an expert agronomist and data scientist specializing in Indian agriculture. Your mission is to create a comprehensive, multilingual knowledge base for an AI-powered advisory chatbot. The target audience is small and marginal farmers in India who may have low digital literacy and speak various regional languages."
    " Therefore, all explanations must be simple, clear, actionable, and easily translatable."
    "Task: Generate a structured knowledge base in Markdown format that covers the following advisory modules. For each module, provide a clear explanation of the advice and generate at least 5 diverse and practical examples in a question-and-answer format that a farmer might ask."
    ,
)

# This is the message with which the system opens the conversation.
WELCOME_MSG = "Welcome to the FarmaBot . Type `q` to quit. How may I serve you today?"

Define a Single turn Chatboot

In [18]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI

# Try using different models. The Gemini 2.0 flash model is highly
# capable, great with tools, and has a generous free tier. If you
# try the older 1.5 models, note that the `pro` models are better at
# complex multi-tool cases like this, but the `flash` models are
# faster and have more free quota.
# Check out the features and quota differences here:
#  - https://ai.google.dev/gemini-api/docs/models/gemini
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")


def chatbot(state: OrderState) -> OrderState:
    """The chatbot itself. A simple wrapper around the model's own chat interface."""
    message_history = [BARISTABOT_SYSINT] + state["messages"]
    return {"messages": [llm.invoke(message_history)]}


# Set up the initial graph based on our state definition.
graph_builder = StateGraph(OrderState)

# Add the chatbot function to the app graph as a node called "chatbot".
graph_builder.add_node("chatbot", chatbot)

# Define the chatbot node as the app entrypoint.
graph_builder.add_edge(START, "chatbot")

chat_graph = graph_builder.compile()

DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

In [13]:
from pprint import pprint

user_msg = "Hello, what can you do?"
state = chat_graph.invoke({"messages": [user_msg]})

# The state object contains lots of information. Uncomment the pprint lines to see it all.
# pprint(state)

# Note that the final state now has 2 messages. Our HumanMessage, and an additional AIMessage.
for msg in state["messages"]:
    print(f"{type(msg).__name__}: {msg.content}")

NameError: name 'chat_graph' is not defined

In [ ]:
user_msg = "Oh great, what kinds of soil does Rice plantation require?"

state["messages"].append(user_msg)
state = chat_graph.invoke(state)

# pprint(state)
for msg in state["messages"]:
    print(f"{type(msg).__name__}: {msg.content}")

HumanMessage: Hello, what can you do?
AIMessage: Hi there! I'm BaristaBot, ready to take your coffee order. I can tell you about our menu items, answer your questions, and put together your order. What are you in the mood for today?
HumanMessage: Oh great, what kinds of latte can you make?
AIMessage: We have several delicious lattes on our menu! There's the classic Latte, the sweet Vanilla Latte, the rich Caramel Latte, and the spicy Cinnamon Latte. We also offer an Iced Latte version of the classic.


Add a Human Node

In [ ]:
from langchain_core.messages.ai import AIMessage


def human_node(state: OrderState) -> OrderState:
    """Display the last model message to the user, and receive the user's input."""
    last_msg = state["messages"][-1]
    print("Model:", last_msg.content)

    user_input = input("User: ")

    # If it looks like the user is trying to quit, flag the conversation
    # as over.
    if user_input in {"q", "quit", "exit", "goodbye"}:
        state["finished"] = True

    return state | {"messages": [("user", user_input)]}


def chatbot_with_welcome_msg(state: OrderState) -> OrderState:
    """The chatbot itself. A wrapper around the model's own chat interface."""

    if state["messages"]:
        # If there are messages, continue the conversation with the Gemini model.
        new_output = llm.invoke([BARISTABOT_SYSINT] + state["messages"])
    else:
        # If there are no messages, start with the welcome message.
        new_output = AIMessage(content=WELCOME_MSG)

    return state | {"messages": [new_output]}


# Start building a new graph.
graph_builder = StateGraph(OrderState)

# Add the chatbot and human nodes to the app graph.
graph_builder.add_node("chatbot", chatbot_with_welcome_msg)
graph_builder.add_node("human", human_node)

# Start with the chatbot again.
graph_builder.add_edge(START, "chatbot")

# The chatbot will always go to the human next.
graph_builder.add_edge("chatbot", "human");

In [20]:
from typing import Literal


def maybe_exit_human_node(state: OrderState) -> Literal["chatbot", "__end__"]:
    """Route to the chatbot, unless it looks like the user is exiting."""
    if state.get("finished", False):
        return END
    else:
        return "chatbot"


graph_builder.add_conditional_edges("human", maybe_exit_human_node)

chat_with_human_graph = graph_builder.compile()

NameError: name 'graph_builder' is not defined

Uncomment state to chat

In [ ]:
# The default recursion limit for traversing nodes is 25 - setting it higher means
# you can try a more complex order with multiple steps and round-trips (and you
# can chat for longer!)
config = {"recursion_limit": 100}

# Remember that this will loop forever, unless you input `q`, `quit` or one of the
# other exit terms defined in `human_node`.
# Uncomment this line to execute the graph:
# state = chat_with_human_graph.invoke({"messages": []}, config)

# Things to try:
#  - Just chat! There's no ordering or menu yet.
#  - 'q' to exit.

# pprint(state)